Refining an already OK approximation of a Nash equilibrium set for the game, Spuds

In [ ]:
using Random
using DataFrames
using CSV
using Statistics
using Printf
using LinearAlgebra

struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
end

const MXS = 100
const MXV = 20
const MNV = 1

function cost(h::Int64, f::Int64, l::Int64, p::Int64, r::Int64, s::Int64)::Int64
    return h*(f+l) + f*(p+r+s)
end

function eval_battle(a::Spud, b::Spud)::Int
    a_finds = a.f >= b.h
    b_finds = b.f >= a.h
    #score_a = 4 * (1000*a.p + a.l > 1000*b.p+b.l) + 3 * (1000*a.r+a.l > 1000*b.r+b.l) + 2 * (1000*a.s+a.l > 1000*b.s+b.l)
    #score_b = 4 * (1000*a.p + a.l < 1000*b.p+b.l) + 3 * (1000*a.r+a.l < 1000*b.r+b.l) + 2 * (1000*a.s+a.l < 1000*b.s+b.l)
    score_a = 4 * (a.p > b.p) + 3 * (a.r > b.r) + 2 * (a.s > b.s)
    score_b = 4 * (a.p < b.p) + 3 * (a.r < b.r) + 2 * (a.s < b.s)
    melee_win = sign(score_a - score_b)
    if a_finds && b_finds
        tiebreaker = 4 * melee_win + 2 * sign(a.l - b.l) + sign(a.h - b.h)
        return sign(tiebreaker)
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        tiebreaker = 4 * sign(a.l - b.l) + 2 * melee_win + sign(a.f - b.f)
        return sign(tiebreaker)
    end
end

tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)
jobs = CSV.read("jobs.csv", DataFrame)

const mult_noun = 1
const mult_adj = 2
const mult_job = 3


function random_name_and_stat()::Spud
    vp = [0, 0, 0, 0, 0, 0]
    #nametype = rand([1,1,1,1,1,2,2,2,3])
    name = ""
    noun = ""
    adj = ""
    noun_i = rand(1:nrow(nouns))
    noun = nouns[noun_i, :noun]
    vp[1] = vp[1] + nouns[noun_i, :H] * mult_noun
    vp[2] = vp[2] + nouns[noun_i, :F] * mult_noun
    vp[3] = vp[3] + nouns[noun_i, :L] * mult_noun
    vp[4] = vp[4] + nouns[noun_i, :P] * mult_noun
    vp[5] = vp[5] + nouns[noun_i, :R] * mult_noun
    vp[6] = vp[6] + nouns[noun_i, :S] * mult_noun
    adj_i = rand(1:nrow(adjectives))
    adj = adjectives[adj_i, :adjective]
    vp[1] = vp[1] + adjectives[adj_i, :H] * mult_adj
    vp[2] = vp[2] + adjectives[adj_i, :F] * mult_adj
    vp[3] = vp[3] + adjectives[adj_i, :L] * mult_adj
    vp[4] = vp[4] + adjectives[adj_i, :P] * mult_adj
    vp[5] = vp[5] + adjectives[adj_i, :R] * mult_adj
    vp[6] = vp[6] + adjectives[adj_i, :S] * mult_adj
    job_i = rand(1:nrow(jobs))
    job = jobs[job_i, :job]
    vp[1] = vp[1] + jobs[job_i, :H] * mult_job
    vp[2] = vp[2] + jobs[job_i, :F] * mult_job
    vp[3] = vp[3] + jobs[job_i, :L] * mult_job
    vp[4] = vp[4] + jobs[job_i, :P] * mult_job
    vp[5] = vp[5] + jobs[job_i, :R] * mult_job
    vp[6] = vp[6] + jobs[job_i, :S] * mult_job
    name = string(adj, " ", job, " ", noun)
    Spud(name, vp[1], vp[2], vp[3], vp[4], vp[5], vp[6])
end

function rand_rename(a::Spud, n_tries::Int = 100)::Spud
    best_score = 0.0
    best_b = random_name_and_stat()
    for ii in 1:n_tries
        b = random_name_and_stat()
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.p^2 + b.r^2 + b.s^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.p * b.p + a.r * b.r + a.s * b.s)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Spud(best_b.name, a.h, a.f, a.l, a.p, a.r, a.s)
end

function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss)
    return df
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :r], df[i, :p], df[i, :s])
    end
    return as
end

function upgrade_spud(sp::Spud)::Spud
    h = sp.h
    f = sp.f
    l = sp.l
    p = sp.p
    r = sp.r
    s = sp.s    
    check_h = (h == MXV) || (cost(h+1,f,l,p,r,s) > MXS)
    check_f = (f == MXV) || (cost(h,f+1,l,p,r,s) > MXS)
    check_l = (l == MXV) || (cost(h,f,l+1,p,r,s) > MXS)
    check_prs = (p+r+s == 3*MXV) || (cost(h,f,l,p+1,r,s) > MXS)
    while !(check_h && check_f && check_l && check_prs)
        rand_i = rand(1:4)
        if rand_i == 1 && !check_h
            h = h+1
        end
        if rand_i == 2 && !check_f
            f = f+1
        end
        if rand_i == 3 && !check_l
            l = l+1
        end
        if rand_i == 4 && !check_prs
            rand_j = rand([1,1,1,1,1,2,2,2,2,3,3,3])
            if rand_j == 1 && p <= MXV
                p += 1
            end
            if rand_j == 2 && r <= MXV
                r += 1
            end
            if rand_j == 3 && s <= MXV
                s += 1
            end
        end            
        check_h = (h == MXV) || (cost(h+1,f,l,p,r,s) > MXS)
        check_f = (f == MXV) || (cost(h,f+1,l,p,r,s) > MXS)
        check_l = (l == MXV) || (cost(h,f,l+1,p,r,s) > MXS)
        check_prs = (p+r+s == 3*MXV) || (cost(h,f,l,p+1,r,s) > MXS)
    end
    return Spud(sp.name, h, f, l, p, r, s)
end



function pick_best_library(bs::Array{Spud})::Spud
    bestscore = -999
    bestf = Spud("",MNV,MNV,MNV,MNV,MNV,MNV)
    df = spuds_to_df(bs)
    hrange = vcat([MNV], df.h, df.h .+ 1, df.f, df.f .+ 1)
    frange = vcat([MNV], df.f, df.f .+ 1, df.h, df.h .+ 1)
    lrange = vcat([MNV], df.l, df.l .+ 1)
    prange = vcat([MNV], df.p, df.p .+ 1)
    rrange = vcat([MNV], df.r, df.r .+ 1)
    srange = vcat([MNV], df.s, df.s .+ 1)
    hrange = sort(unique(hrange))
    frange = sort(unique(frange))
    lrange = sort(unique(lrange))
    prange = sort(unique(prange))
    rrange = sort(unique(rrange))
    srange = sort(unique(srange))
    for h in hrange
        if (cost(h, MNV, MNV, MNV, MNV, MNV) <= MXS)
            for f in frange
                if (cost(h, f, MNV, MNV, MNV, MNV) <= MXS)
                    for l in lrange
                        if (cost(h, f, l, MNV, MNV, MNV) <= MXS)
                            for p in prange
                                if (cost(h, f, l, p, MNV, MNV) <= MXS)
                                    for r in rrange
                                        if (cost(h, f, l, p, r, MNV) <= MXS)
                                            for s in srange
                                                if (cost(h,f,l,p,r,s) <= MXS)
                                                    ff = Spud("",h,f,l,p,r,s)
                                                    score = eval_battle_list(ff, bs)
                                                    if score > bestscore
                                                        bestscore = score
                                                        bestf = ff
                                                    end
                                                end
                                            end
                                        end
                                    end                        
                                end
                            end                        
                        end
                    end
                end
            end
        end
    end
    return upgrade_spud(bestf)
end


function pick_best_library2(bs::Array{Spud}, w::Vector{Float64})::Spud
    bestscore = -999.9
    bestf = Spud("",MNV,MNV,MNV,MNV,MNV,MNV)
    df = spuds_to_df(bs)
    hrange = vcat([MNV], df.h, df.h .+ 1, df.f, df.f .+ 1)
    frange = vcat([MNV], df.f, df.f .+ 1, df.h, df.h .+ 1)
    lrange = vcat([MNV], df.l, df.l .+ 1)
    prange = vcat([MNV], df.p, df.p .+ 1)
    rrange = vcat([MNV], df.r, df.r .+ 1)
    srange = vcat([MNV], df.s, df.s .+ 1)
    hrange = sort(unique(hrange))
    frange = sort(unique(frange))
    lrange = sort(unique(lrange))
    prange = sort(unique(prange))
    rrange = sort(unique(rrange))
    srange = sort(unique(srange))
    for h in hrange
        if (cost(h, MNV, MNV, MNV, MNV, MNV) <= MXS)
            for f in frange
                if (cost(h, f, MNV, MNV, MNV, MNV) <= MXS)
                    for l in lrange
                        if (cost(h, f, l, MNV, MNV, MNV) <= MXS)
                            for p in prange
                                if (cost(h, f, l, p, MNV, MNV) <= MXS)
                                    for r in rrange
                                        if (cost(h, f, l, p, r, MNV) <= MXS)
                                            for s in srange
                                                if (cost(h,f,l,p,r,s) <= MXS)
                                                    ff = Spud("",h,f,l,p,r,s)
                                                    score = eval_battle_list2(ff, bs, w) + 0.001 * rand()
                                                    if score > bestscore
                                                        bestscore = score
                                                        bestf = ff
                                                    end
                                                end
                                            end
                                        end
                                    end                        
                                end
                            end                        
                        end
                    end
                end
            end
        end
    end
    return upgrade_spud(bestf)
end

function ffp(nash_env, nits)
    n_nash = length(nash_env)
    i_lose = Array{Int}(undef, (n_nash, n_nash))
    n_lose = Array{Int}(undef, n_nash)
    for i in 1:n_nash
        n_lose[i] = 0
        ff = nash_env[i]
        for j in 1:n_nash
            if eval_battle(ff, nash_env[j]) ==-1
                n_lose[i] += 1
                i_lose[i, n_lose[i]] = j
            end
        end
    end
    counts = [0 for i in 1:n_nash]
    wins = [0 for i in 1:n_nash]
    for i in 1:n_nash
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    for iter in 1:nits
        wc = 2 .* wins .+ counts
        ind_winners = findall(wc .== maximum(wc))
        i = rand(ind_winners)
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    return counts
end

function cost(a::Spud)::Int64
    return cost(a.h, a.f, a.l, a.p, a.r, a.s)
end

In [ ]:
function subset_library(bs::Array{Spud}, n_hits::Int, w::Vector{Float64}, thres::Float64)::Array{Spud}
    bestf = Array{Spud}(undef, n_hits)
    evs = Vector{Float64}(undef, n_hits)
    spud_i = 0
    df = spuds_to_df(bs)
    best_score = 0.0
    hrange = vcat([MNV], df.h, df.h .+ 1, df.f, df.f .+ 1)
    frange = vcat([MNV], df.f, df.f .+ 1, df.h, df.h .+ 1)
    lrange = vcat([MNV], df.l, df.l .+ 1)
    prange = vcat([MNV], df.p, df.p .+ 1)
    rrange = vcat([MNV], df.r, df.r .+ 1)
    srange = vcat([MNV], df.s, df.s .+ 1)
    hrange = sort(unique(hrange))
    frange = sort(unique(frange))
    lrange = sort(unique(lrange))
    prange = sort(unique(prange))
    rrange = sort(unique(rrange))
    srange = sort(unique(srange))
    overflow = 0
    for h in hrange
        if (cost(h, MNV, MNV, MNV, MNV, MNV) <= MXS)
            for f in frange
                if (cost(h, f, MNV, MNV, MNV, MNV) <= MXS)
                    for l in lrange
                        if (cost(h, f, l, MNV, MNV, MNV) <= MXS)
                            for p in prange
                                if (cost(h, f, l, p, MNV, MNV) <= MXS)
                                    for r in rrange
                                        if (cost(h, f, l, p, r, MNV) <= MXS)
                                            for s in srange
                                                if (cost(h,f,l,p,r,s) <= MXS)
                                                    ff = Spud("",h,f,l,p,r,s)
                                                    score = eval_battle_list2(ff, bs, w)
                                                    if score > best_score
                                                        best_score = score
                                                    end
                                                    if score >= (best_score - thres)
                                                        # find the first spud_i with ev less than best_score - thres
                                                        orig_spud_i = spud_i
                                                        spud_i += 1
                                                        if spud_i > n_hits
                                                            spud_i = 1
                                                            overflow = 1
                                                        end
                                                        while (evs[spud_i] >= best_score - thres && spud_i != orig_spud_i)
                                                            spud_i += 1
                                                            if spud_i > n_hits
                                                                spud_i = 1
                                                                overflow = 1
                                                            end
                                                        end
                                                        if spud_i == orig_spud_i
                                                            if overflow==1
                                                                return unique(bestf[evs .>= best_score - thres])
                                                            else
                                                                bestf = bestf[1:spud_i]
                                                                evs = evs[1:spud_i]
                                                                return unique(bestf[evs .>= best_score - thres])
                                                            end
                                                        else
                                                            bestf[spud_i] = upgrade_spud(ff)
                                                            evs[spud_i] = score
                                                        end
                                                    end
                                                end
                                            end
                                        end
                                    end                        
                                end
                            end                        
                        end
                    end
                end
            end
        end
    end
    println(best_score)
    if overflow==1
        return unique(bestf[evs .>= best_score - thres])
    else
        bestf = bestf[1:spud_i]
        evs = evs[1:spud_i]
        return unique(bestf[evs .>= best_score - thres])
    end
end

In [ ]:
ne = nash_env[1:100 .* 2]
counts = ffp(ne, 100000)
lib = subset_library(ne, 900, counts./sum(counts), 0.05)
[eval_battle_list2(ff, ne, counts./sum(counts)) for ff in lib]

In [ ]:
ff = lib[3]
eval_battle_list2(ff, ne, counts./sum(counts))

In [ ]:
cost(lib[3])

In [ ]:
[cost(ff) for ff in nash_env]

In [ ]:
lib

In [ ]:
ff = pick_best_library2(ne, counts./sum(counts))

In [ ]:
eval_battle_list2(ff, ne, counts./sum(counts))

In [ ]:
nash_env_df = DataFrame(CSV.File("spuds_nash_stage04.csv"))
nash_env = df_to_spuds(nash_env_df)

In [ ]:
counts = ffp(nash_env, 100000)
println(maximum([eval_battle_list2(ff, nash_env, counts./sum(counts)) for ff in nash_env]))